**NOTEBOOK MODIFIED BY MUHAMMAD ASAD IQBAL**

The main aim of this notebook is to demonstrate how to perform various data preprocessing, exploratory data analysis (EDA), feature engineering, and machine learning tasks using PySpark, a Python library for distributed data processing and machine learning. Specifically, the notebook focuses on working with a Titanic dataset, which contains passenger information (such as age, sex, class, and fare) and whether they survived the Titanic disaster or not. The key objectives of this notebook can be summarized as follows:

#Data Preprocessing:
The notebook starts by showing how to set up PySpark, create a Spark session, and read CSV files. It emphasizes the importance of defining the schema when reading data, as Spark requires schema definition. The notebook demonstrates how to handle missing data and convert data types when necessary.

#Exploratory Data Analysis (EDA):
The notebook then proceeds to perform EDA on the dataset. It uses various Spark operations to explore and analyze the data. This includes inspecting data schema, displaying sample rows, calculating summary statistics, and generating tabular visualizations to gain insights into the dataset.

#Feature Engineering:
Feature engineering involves creating new features or modifying existing ones to improve the predictive power of a machine learning model. In this notebook, feature engineering tasks include creating a "FamilySize" feature based on the number of family members, converting categorical variables (e.g., "Sex" and "Embarked") into numerical categories, and imputing missing values, particularly for the "Age" feature.

#Machine Learning Model Building:
The notebook demonstrates how to build machine learning models using Spark MLlib. It covers both linear models (Logistic Regression with Ridge and Lasso regularization) and ensemble tree-based models (Random Forest and Gradient Boosting). For each model, it emphasizes the need to specify the label column and feature columns.

#Model Evaluation:
After training the machine learning models, the notebook evaluates their performance using accuracy as the metric. It utilizes Spark's MulticlassClassificationEvaluator to assess the accuracy of the models on a validation dataset.

#Prediction:
The notebook also hints at making predictions on a test dataset and mentions the need to preprocess the test data in a similar way to the training data. The final aim is typically to make predictions for passenger survival on the Titanic test data and possibly submit the results.



# Starting pySpark

Spark is not available in Kaggle notebook by default. But installing pySpark in Kaggle kernel is easy.

In [1]:
pip install pyspark

First, we need to start a SparkSession and create a spark instance.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('classification').getOrCreate()

In [3]:
from itertools import chain
from pyspark.sql.functions import count, mean, when, lit, create_map, regexp_extract

In [4]:
df1 = spark.read.csv('train.csv',\
                     header=True, inferSchema=True)
df2 = spark.read.csv('test.csv', \
                     header=True, inferSchema=True)

We can view the schema here. There is no need to print the column names separately. Although this option is available in Spark. We will use it shortly for a different purposes.

In [5]:
df1.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Pandas has head(), tail(), sample() options to view few rows of the dataframe. Spark dataframe also has all of those options which you can try. But here we use show(n) method to view sample rows of the dataframe. By default show() presents 20 rows. Please remember the discussion of transformation vs action above. All of these are example of Spark action.

In [6]:
df1.show(4)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 4 rows



The output of the show() might look ugly, especially if there are a large number of columns in the dataframe. At this point, we might miss Pandas head(). There is an option to convert the Spark dataframe into the Pandas dataframe. But we have to be careful here. Usually, Spark is handling a large volume of data, and converting it to Pandas stores everything immediately to the memory. Which we should avoid for the large data. However, there is a way out. Please remember the lazy evaluation of the Spark transformation. We can transform the Spark dataframe by limit(n) to take only n number of rows and then convert that to the Pandas. toPandas() is an action.

In [7]:
df1.limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


Alternatively we can select() a few columns and inspect within Spark. select() is an example of Spark transformation. Therefore that step is evaluated lazily. Hence we pass a Spark action show() at the end to print the result.

In [8]:
df1.select('Survived', 'Pclass', 'Age', 'Fare').show(4)

+--------+------+----+-------+
|Survived|Pclass| Age|   Fare|
+--------+------+----+-------+
|       0|     3|22.0|   7.25|
|       1|     1|38.0|71.2833|
|       1|     3|26.0|  7.925|
|       1|     1|35.0|   53.1|
+--------+------+----+-------+
only showing top 4 rows



Spark has describe() method similar to the Pandas. But I find a summary() method more versatile than describe(). Please check [here](https://github.com/roshankoirala/pySpark_tutorial/blob/master/Exploratory_data_analysis_with_pySpark.ipynb) for detail. Both describe() and summary() are Spark trasnformations. Therefore, they do not produce result immediately. Hence we need show() at the end.

In [9]:
df1.select('Survived', 'Pclass', 'Age', 'Fare').summary().show()

+-------+-------------------+------------------+------------------+-----------------+
|summary|           Survived|            Pclass|               Age|             Fare|
+-------+-------------------+------------------+------------------+-----------------+
|  count|                891|               891|               714|              891|
|   mean| 0.3838383838383838| 2.308641975308642| 29.69911764705882| 32.2042079685746|
| stddev|0.48659245426485753|0.8360712409770491|14.526497332334035|49.69342859718089|
|    min|                  0|                 1|              0.42|              0.0|
|    25%|                  0|                 2|              20.0|           7.8958|
|    50%|                  0|                 3|              28.0|          14.4542|
|    75%|                  1|                 3|              38.0|             31.0|
|    max|                  1|                 3|              80.0|         512.3292|
+-------+-------------------+------------------+------

count() acts differently in Pandas and Spark. In Spark, it gives the total number of rows in the dataframe. There is no direct way to find the shape of the dataframe. We can use the following trick.  Here count and columns are action.

In [10]:
print('Number of rows: \t', df1.count())
print('Number of columns: \t', len(df1.columns))

Number of rows: 	 891
Number of columns: 	 12


# Exploratory Data Analysis

### About the data visualization in the Spark  

There is no native visualization library in Spark. But we can do the lazy transformation on the dataframe, extract the necessary numbers, and make the visualizations out of that. The implementation of this idea can be found [here](https://github.com/roshankoirala/pySpark_tutorial/blob/master/Data_visualization_in_pySpark%20.ipynb).

There are options to make visualization by extending other libraries though. However, we do not go to that route here. We will focus on tabular visualization. Tabular visualization is not a bad option.

### How many people survived?

We can use groupby() and count() transformations to do that. Both are lazy transformation. Again remember that the Spark transformation alone don't evaluate things unless we call an action upon them. Here show() is an action to print the results.

In [11]:
df1.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



### Continious variables

Among the features, Fare and Age are the continuous variables (non-categorical). We can inspect them closely here. Our interest would be to find average fare and age. We already use summary() to calculate mean. If we just want mean we can either to summary('mean') or we can also directly call mean() and select columns inside that. Also in summary we can pass multiple arguments like df.summary('mean', 'stddev') and so on. Again please refer to [this](https://github.com/roshankoirala/pySpark_tutorial/blob/master/Exploratory_data_analysis_with_pySpark.ipynb) link for detail. Here groupby() and mean() are Spark transformation. Now you probably started to figure out which is transformation and which is action. I will stop iterating that in every single case now on.

Passenger paying more money for the fair is likely to survive than those paying less. This variable might have collinearity with the passenger class that we investigate later. Age seems to be not that important for survival compared to fare.

In [12]:
df1.groupBy('Survived').mean('Fare', 'Age').show()

+--------+------------------+------------------+
|Survived|         avg(Fare)|          avg(Age)|
+--------+------------------+------------------+
|       1| 48.39540760233917|28.343689655172415|
|       0|22.117886885245877| 30.62617924528302|
+--------+------------------+------------------+



### Categorical variables

Here we see how each of the categorical variables has affected the survival of the passenger. Spark dataframe also has a pivot() method very similar to the Pandas dataframe to perform this task.

Below we see that sex is an (probably the most) important factor for survival. The survival ratio of the female is much higher than that of male.

In [13]:
df1.groupBy('Survived').pivot('Sex').count().show()

+--------+------+----+
|Survived|female|male|
+--------+------+----+
|       1|   233| 109|
|       0|    81| 468|
+--------+------+----+



Similarly, the first-class passengers are more likely to survive than the second class. And the third class passengers had very hard luck.

In [14]:
df1.groupBy('Survived').pivot('Pclass').count().show()

+--------+---+---+---+
|Survived|  1|  2|  3|
+--------+---+---+---+
|       1|136| 87|119|
|       0| 80| 97|372|
+--------+---+---+---+



The number of siblings and the number of parents also play some role in their survival. The large family is less likely to survive. Similarly, the person with no companion is also less likely to survive.


In [15]:
df1.groupBy('Survived').pivot('SibSp').count().show()

+--------+---+---+---+---+---+----+----+
|Survived|  0|  1|  2|  3|  4|   5|   8|
+--------+---+---+---+---+---+----+----+
|       1|210|112| 13|  4|  3|NULL|NULL|
|       0|398| 97| 15| 12| 15|   5|   7|
+--------+---+---+---+---+---+----+----+



In [16]:
df1.groupBy('Survived').pivot('Parch').count().show()

+--------+---+---+---+---+----+---+----+
|Survived|  0|  1|  2|  3|   4|  5|   6|
+--------+---+---+---+---+----+---+----+
|       1|233| 65| 40|  3|NULL|  1|NULL|
|       0|445| 53| 40|  2|   4|  4|   1|
+--------+---+---+---+---+----+---+----+



Embark also seems to be important. But it can be collinear with Pclass and fair.

In [17]:
df1.groupBy('Survived').pivot('Embarked').count().show()

+--------+----+---+---+---+
|Survived|null|  C|  Q|  S|
+--------+----+---+---+---+
|       1|   2| 93| 30|217|
|       0|NULL| 75| 47|427|
+--------+----+---+---+---+



# Feature Engineering

First, let's see if there are any missed data.

In [18]:
for col in df1.columns:
    print(col.ljust(20), df1.filter(df1[col].isNull()).count())

PassengerId          0
Survived             0
Pclass               0
Name                 0
Sex                  0
Age                  177
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin                687
Embarked             2


There are many Cabin info is missing. The Cabin is related to Pclass. We will drop this feature. So no problem so far. There are, 2 entries of Embarked missing. We will fill it with the most repeated value S. Age of many people is missing. Again the simplest way to impute the age would be to fill by the average. We choose median for fare imputation. We use Spark's fillna() method to do that. For age we use more complex imputation method discussed below. For now I am just focusing on the train data. There can be different feature missing in the test data. Acutally there is missed fair in test data. So we calculate median fair also. We come to the test data at the end of this notebook.

In [19]:
df1.select('Fare', 'Embarked').summary('mean', '50%', 'max').show()

+-------+----------------+--------+
|summary|            Fare|Embarked|
+-------+----------------+--------+
|   mean|32.2042079685746|    NULL|
|    50%|         14.4542|    NULL|
|    max|        512.3292|       S|
+-------+----------------+--------+



In [20]:
df1 = df1.fillna({'Embarked': 'S', 'Fare':14.45})

The basic idea for age imputation is to take the title of the people from the name column and impute with the average age of the group of people with that title. Mrs tend to be older than Miss. This method originally appeared in [this](https://www.kaggle.com/konstantinmasich/titanic-0-82-0-83) kernel. We will present the pySpark version of the implementation.

First, we extract the title using the regular expression and observe the count and average age with each of the titles.

In [21]:
df1 = df1.withColumn('Title', regexp_extract(df1['Name'],\
                '([A-Za-z]+)\.', 1))

df1.groupBy('Title').agg(count('Age'), mean('Age')).sort('count(Age)').show()

+--------+----------+------------------+
|   Title|count(Age)|          avg(Age)|
+--------+----------+------------------+
|     Don|         1|              40.0|
|Countess|         1|              33.0|
|    Lady|         1|              48.0|
|     Mme|         1|              24.0|
|    Capt|         1|              70.0|
|     Sir|         1|              49.0|
|Jonkheer|         1|              38.0|
|      Ms|         1|              28.0|
|     Col|         2|              58.0|
|    Mlle|         2|              24.0|
|   Major|         2|              48.5|
|     Rev|         6|43.166666666666664|
|      Dr|         6|              42.0|
|  Master|        36| 4.574166666666667|
|     Mrs|       108|35.898148148148145|
|    Miss|       146|21.773972602739725|
|      Mr|       398|32.368090452261306|
+--------+----------+------------------+



It is seen that Mr, Miss, and Mrs are highly repeated than other titles. The count of Master is not that high but its average age is much lower than others. So we keep those four titles and map other with one of the first three.

In [22]:
title_dic = {'Mr':'Mr', 'Miss':'Miss', 'Mrs':'Mrs', 'Master':'Master', \
             'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',\
             'Don': 'Mr', 'Mme': 'Miss', 'Jonkheer': 'Mr', 'Lady': 'Mrs',\
             'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs', \
             'Dr':'Mr', 'Rev':'Mr'}

mapping = create_map([lit(x) for x in chain(*title_dic.items())])

df1 = df1.withColumn('Title', mapping[df1['Title']])
df1.groupBy('Title').mean('Age').show()

+------+------------------+
| Title|          avg(Age)|
+------+------------------+
|  Miss|             21.86|
|Master| 4.574166666666667|
|    Mr| 33.02272727272727|
|   Mrs|35.981818181818184|
+------+------------------+



Now we create a function that imputes the age column with the average age of the group of people having the same name title as theirs. And use it to impute the ages in the next stage.

In [23]:
def age_imputer(df, title, age):

    '''This function search for the null in 'Age' column
    of the dataframe df. If there is null then it look
    for the title and fill the 'Age' with age argument.
    If 'Age' is not null, it will keep the same age.  '''

    return df.withColumn('Age', \
                         when((df['Age'].isNull()) & (df['Title']==title), \
                              age).otherwise(df['Age']))

In [24]:
df1 = age_imputer(df1, 'Mr', 33.02)
df1 = age_imputer(df1, 'Mrs', 35.98)
df1 = age_imputer(df1, 'Miss', 21.86)
df1 = age_imputer(df1, 'Master', 4.75)

### Creating a new column and dropping a column

Now we create a new column called FamilySize combining Parch and SibSp. This API is significantly different in Spark than in Pandas. We use withColumn() method to do that. The first input in the method is a string of the name of the new column. This creates a new column and also keeps the old columns. We will drop the Parch and SibSp column afterward.

In [25]:
df1 = df1.withColumn('FamilySize', df1['Parch'] + df1['SibSp']).\
            drop('Parch', 'SibSp')

And drop the unwanted columns.

In [26]:
df1 = df1.drop('PassengerID', 'Cabin', 'Name', 'Ticket', 'Title')

Now we have a trimmed dataframe. For the small size dataframe show(n) method is not that worse than Pandas head(). See that Sex and Embarked columns are strings. We have to convert them to numeric categories.

In [27]:
df1.show(4)

+--------+------+------+----+-------+--------+----------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|FamilySize|
+--------+------+------+----+-------+--------+----------+
|       0|     3|  male|22.0|   7.25|       S|         1|
|       1|     1|female|38.0|71.2833|       C|         1|
|       1|     3|female|26.0|  7.925|       S|         0|
|       1|     1|female|35.0|   53.1|       S|         1|
+--------+------+------+----+-------+--------+----------+
only showing top 4 rows



And there is no missing value now.

In [28]:
for col in df1.columns:
    print(col.ljust(20), df1.filter(df1[col].isNull()).count())

Survived             0
Pclass               0
Sex                  0
Age                  0
Fare                 0
Embarked             0
FamilySize           0


# Model building

So far we used Spark dataframe available in Spark SQL for EDA and feature engineering. Now we will use the Spark ML library to do ML tasks. We will cover the following ML task on Spark ML here:

- StringIndexer: Converts string categories to numerical categories.
- Vector Assembler: Special to Spark API. We will find detail shortly.
- Logistic regression based on Ridge and Lasso regularization.
- Tree-based ensemble methods: Random forest and Gradient boosting.
- Pipeline: It is a big deal for big data.


In [29]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression,\
                    RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

### String Indexer

We will convert the Sex and Embarked column from string to numeric index. This creates a new column for numeric leaving the original intact. So we will remove them afterward.

In [30]:
stringIndex = StringIndexer(inputCols=['Sex', 'Embarked'],
                       outputCols=['SexNum', 'EmbNum'])

stringIndex_model = stringIndex.fit(df1)

df1_ = stringIndex_model.transform(df1).drop('Sex', 'Embarked')
df1_.show(4)

+--------+------+----+-------+----------+------+------+
|Survived|Pclass| Age|   Fare|FamilySize|SexNum|EmbNum|
+--------+------+----+-------+----------+------+------+
|       0|     3|22.0|   7.25|         1|   0.0|   0.0|
|       1|     1|38.0|71.2833|         1|   1.0|   1.0|
|       1|     3|26.0|  7.925|         0|   1.0|   0.0|
|       1|     1|35.0|   53.1|         1|   1.0|   0.0|
+--------+------+----+-------+----------+------+------+
only showing top 4 rows



### What is VectorAssembler?

In Python's scikit learn API the model takes X and y variable in the separation matrix. The target y is usually a column vector and feature X is a matrix. scikit learn accepts X as a matrix of dataframe directly. But Spark API is different here. First, it requires X and y in a single matrix instead of two for the training data. It accepts X only in the prediction part as it should. And also X should be a vector in each row (see the output below) of the dataframe. In short, we can not directly feed the dataframe in the model. We should do what VectorAssembler does.

In below, inputCols are the feature columns that are doing to be merged to make a vector in each row and outputCol is the name of the merged column. This is the column that Spark ML identifies as the feature column. It is a common practice to rename this as features as Spark ML identifies this name. If its name is different you have to mention column name when fitting model. Then we can select only the feature column and y column. See the illustration below.

In [31]:
vec_asmbl = VectorAssembler(inputCols=df1_.columns[1:],
                           outputCol='features')

df1_ = vec_asmbl.transform(df1_).select('features', 'Survived')
df1_.show(4, truncate=False)

+------------------------------+--------+
|features                      |Survived|
+------------------------------+--------+
|[3.0,22.0,7.25,1.0,0.0,0.0]   |0       |
|[1.0,38.0,71.2833,1.0,1.0,1.0]|1       |
|[3.0,26.0,7.925,0.0,1.0,0.0]  |1       |
|[1.0,35.0,53.1,1.0,1.0,0.0]   |1       |
+------------------------------+--------+
only showing top 4 rows



Now we split the training data into the train and validation part. We split the data into a 7:3 ratio.

In [32]:
train_df, valid_df = df1_.randomSplit([0.7, 0.3])

In [33]:
train_df.show(4, truncate=False)

+---------------------+--------+
|features             |Survived|
+---------------------+--------+
|(6,[0,1],[1.0,33.02])|0       |
|(6,[0,1],[1.0,33.02])|0       |
|(6,[0,1],[1.0,38.0]) |0       |
|(6,[0,1],[1.0,39.0]) |0       |
+---------------------+--------+
only showing top 4 rows



In this output form [0] in the bracket can be confusing. 5, [0] means five consecutive entries are zero. Split has not split the data column-wise.


### Linear model

We study logistic regression here. Spark ML offers elastic net regularization by default. The regularization function is given by

$$ \alpha (\lambda | {\bf{w}} |_1) + (1 - \alpha) \left(\frac\lambda2 |{\bf{w}}|_2^2 \right) $$

In spark API $\alpha$ is eleasticNetParam and $\lambda$ is regParam. We can make our model Ridge by choosing $\alpha=0$ and Lasso by choosing $\alpha=1$.

Please note that we need to specify the label column at this stage. It the feature column was named differently we had to specify that too here. In Spark, we fit() the model similar to scikit learn but unlike scikit learn we need to name the fitted instance (see comment below). Then we have unified function evaluate() and we call evaluation parameters like predict, accuracy on the evaluation instance.



### Evaluation and metric


First, we instantiate MulticlassClassificationEvaluator(). We need to specify the metric we want to evaluate at this stage, like metricName='accuracy' in our case. Fitting and evaluating models follow similarly from there. There is an alternative way to evaluate the accuracy scores in the linear model with the following set of commands:

- model_name = model.fit(data)
- pred = model_name.evaluate(data)
- pred.accuracy

In this method, there is no need to import MulticlassClassificationEvaluator(). But we stick with the first convention as it provides uniform API for all the models.


At this point, I will remind the discussion of transformation and action again. All the machine learning models and preprocessing modules in Spark are transformations. They are evaluated lazily. When we ask for prediction of a model or score of the model then these are Spark action. For example MulticlassClassificationEvaluator() is a transformation while evaluate() is an action.

In [34]:
evaluator = MulticlassClassificationEvaluator(labelCol='Survived',
                                          metricName='accuracy')

In [35]:
ridge = LogisticRegression(labelCol='Survived',
                        maxIter=100,
                        elasticNetParam=0, # Ridge regression is choosen
                        regParam=0.03)

model = ridge.fit(train_df)
pred = model.transform(valid_df)
evaluator.evaluate(pred)

0.8108108108108109

In [36]:
lasso = LogisticRegression(labelCol='Survived',
                           maxIter=100,
                           elasticNetParam=1, # Lasso
                           regParam=0.0003)

model = lasso.fit(train_df)
pred = model.transform(valid_df)
evaluator.evaluate(pred)

0.8185328185328186

Generally, Lasso performs better than Ridge.


### Ensemble Tree


Currently Spark ML supports two types of ensemble algorithm. Random forest for bagging and gradient boosting for boosting. There is no stacking algorithm available in Spark ML yet. Here we will study both availabel ensemble method both are tree-based methods.


In [37]:
rf = RandomForestClassifier(labelCol='Survived',
                           numTrees=100, maxDepth=3)

model = rf.fit(train_df)
pred = model.transform(valid_df)
evaluator.evaluate(pred)

0.8223938223938224

In [38]:
gb = GBTClassifier(labelCol='Survived', maxIter=75, maxDepth=3)

model = gb.fit(train_df)
pred = model.transform(valid_df)
evaluator.evaluate(pred)

0.8455598455598455

Generally, the tree-based ensemble method performs better than the linear model and among the tree-based model, gradient boosting performs better than random forest. We may test different method for our final submission.


# Prediction

Now we focus on making a prediction on test data and submit the result. We need to follow the exact same procedure for the test data for data cleaning. First we observer the header and see if there are any missing values in the test data.

In [39]:
df2.show(4)

+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| NULL|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| NULL|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| NULL|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|315154|8.6625| NULL|       S|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 4 rows



In [40]:
for col in df2.columns:
    print(col.ljust(20), df2.filter(df2[col].isNull()).count())

PassengerId          0
Pclass               0
Name                 0
Sex                  0
Age                  86
SibSp                0
Parch                0
Ticket               0
Fare                 1
Cabin                327
Embarked             0


Unlike the train data, there is no missing value in the Embarked column but there is one missing value for fair. And there are few ages missing. Now we fill the missing value by the median fair (of train data, not the test data). We ignore other missing values as we are dropping Cabin from our model. First, we will make a family size feature and drop the unwanted.

In [41]:
df2 = df2.fillna({'Embarked': 'S', 'Fare':14.45})
df2 = df2.withColumn('FamilySize', df2['Parch'] + df2['SibSp']).\
            drop('Parch', 'SibSp')

Now we come to imputing missing age in the test data. We need to follow exactly the same stages as we did in the train data. Only thing we need to be careful is that we are imputing the averages based on the training data but not the test data. You will identify these steps below.

In [42]:
df2 = df2.withColumn('Title', regexp_extract(df2['Name'],\
                '([A-Za-z]+)\.', 1))

df2 = df2.withColumn('Title', mapping[df2['Title']])

df2.groupBy('Title').agg(count('Age'), mean('Age')).sort('count(Age)').show()

+------+----------+------------------+
| Title|count(Age)|          avg(Age)|
+------+----------+------------------+
|Master|        17| 7.406470588235294|
|   Mrs|        63|38.904761904761905|
|  Miss|        64|21.774843750000002|
|    Mr|       188|32.340425531914896|
+------+----------+------------------+



In [43]:
df2 = age_imputer(df2, 'Mr', 33.02)
df2 = age_imputer(df2, 'Mrs', 35.98)
df2 = age_imputer(df2, 'Miss', 21.86)
df2 = age_imputer(df2, 'Master', 4.75)

df2 = df2.drop('Cabin', 'Name', 'Ticket', 'Title') # keep PassengerId
df2.show(4)

+-----------+------+------+----+------+--------+----------+
|PassengerId|Pclass|   Sex| Age|  Fare|Embarked|FamilySize|
+-----------+------+------+----+------+--------+----------+
|        892|     3|  male|34.5|7.8292|       Q|         0|
|        893|     3|female|47.0|   7.0|       S|         1|
|        894|     2|  male|62.0|9.6875|       Q|         0|
|        895|     3|  male|27.0|8.6625|       S|         0|
+-----------+------+------+----+------+--------+----------+
only showing top 4 rows



Let's check the missing values again.

In [44]:
for col in df2.columns:
    print(col.ljust(20), df2.filter(df2[col].isNull()).count())

PassengerId          0
Pclass               0
Sex                  0
Age                  0
Fare                 0
Embarked             0
FamilySize           0


### Pipeline

At this stage, it is worth introducing pipeline. In machine learning, it is common to run a sequence of algorithms to process and learn from data. In our example, we performed StringIndexer, VectorAssembler, and ML model. In other cases, the intermediate stages can be standardization, vectorization (for text processing), normalization, etc. These operations have to be performed on a specific order. Spark represents such a workflow as a Pipeline, which consists of a sequence of stages to be run in a specific order. Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

Without the pipeline, we have to execute each stage, store the outcome, and feed into the next stage and evaluate, and so on. We prefer pipeline over this manual approach because of the following reasons:

- The pipeline is less prone to mistake because the processes are automated.
- In a production environment, this is the only way to do machine learning end to end.
- Pipeline enhances the lazy evaluation. So this is a very natural choice in Spark. The pipeline is even more important for big data.


### Grid-search and cross-validation

Usually, there are many hyperparameters in a model of selection and some combination of those parameters might give the best result. Tuning them requires checking all possible combinations of the hyperparameter. Doing them manually is a tedious bookkeeping task. Fortunately, there is a grid search option available in Spark like in Sci-kit learn.

When doing the grid search we need to validate the model using a separate dataset that was not used to train the data. So far we used customized validation set for comparison between different models. Usually, Spark would be handling very big data. For big data, the train-validation split can be sufficient.  For small datasets like this, however, cross-validation is preferred over the train-validation split. Coss-validation is available in Spark. We will use five-fold cross-validation for better model selection.

We use CrossValidator available in Spark ML for the cross-validation. CrossValidator accepts estimatorParamMaps in which we can pass a grid search object built with ParamGridBuilder which is also available in Spark ML.

We have chosen a random forest for our submission model. We test three hyperparameters from the random forest: number of trees, minimum information gain in each split. The tuning of the number of trees is not that tricky, higher is better. The only concern here is time it takes for a large number of trees taken. The depth of the tree should be tuned properly. Larger depth with some non-zero info gain can give the best performance. Other objects in the model pipeline have no hyperparameters. If they would we could make a grid using those as well.



In [45]:
pipeline_rf = Pipeline(stages=[stringIndex, vec_asmbl, rf])

paramGrid = ParamGridBuilder().\
            addGrid(rf.maxDepth, [3, 4, 5]).\
            addGrid(rf.minInfoGain, [0., 0.01, 0.1]).\
            addGrid(rf.numTrees, [1000]).\
            build()

selected_model = CrossValidator(estimator=pipeline_rf,
                                estimatorParamMaps=paramGrid,
                                evaluator=evaluator,
                                numFolds=5)

model_final = selected_model.fit(df1)
pred_train = model_final.transform(df1)
evaluator.evaluate(pred_train)

0.8507295173961841

This is the in-sample accuracy which is generally higher than the out-sample accuracy.

In [46]:
pred_test = model_final.transform(df2)

predictions = pred_test.select('PassengerId', 'prediction')
predictions = predictions.\
                withColumn('Survived', predictions['prediction'].\
                cast('integer')).drop('prediction')
predictions.show(5)

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|        892|       0|
|        893|       0|
|        894|       0|
|        895|       0|
|        896|       1|
+-----------+--------+
only showing top 5 rows



The following is the method to read csv file in Spark. We can even read the csv file using Spark API. But there is some problem with that, especially Pandas can not read that csv and submission through kernel does not work for it. For this reason we change the submission file to pandas and make a submission.  

In [47]:
# Writing csv file in Spark
predictions.coalesce(1).write.csv('submission_file.csv', header=True)

In [48]:
# Reading the saved file from spark
spark.read.csv('submission_file.csv', header=True).show(4)

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|        892|       0|
|        893|       0|
|        894|       0|
|        895|       0|
+-----------+--------+
only showing top 4 rows



In [49]:
# Writing csv file using Pandas
predictions.toPandas().to_csv('submission.csv', index=False)

In [50]:
# Inspecting csv file in pandas
import pandas as pd
pd.read_csv('submission.csv').head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


We can also save the model itself for future use so that you don't have to train every time.  

In [51]:
model_final.write().save('titanic_classification.model')

In [52]:
! ls titanic_classification.model/*

titanic_classification.model/bestModel:
metadata  stages

titanic_classification.model/estimator:
metadata  stages

titanic_classification.model/evaluator:
metadata

titanic_classification.model/metadata:
part-00000  _SUCCESS


# Final thought.

- We just presented a base model here and established that Spark is basically capable of doing many tasks on machine learning and model building workflow seamlessly. The model is not fine-tuned yet. It would be interesting to see Spark matching Sci-kit learn's performance.

- We did not perform standardization. The reason is standardization is inbuilt in the Spark linear model and it is not needed for the tree-based models. If we had chosen a linear model as our prediction model we may have to turn it off in order to make normalization based on train data instead of test data while making a prediction.

- We did not include everything in the pipeline. For example, we imputed null values outside the pipeline. In a production environment, it is required to have everything in the pipeline. So there is room for improvement.